<a href="https://colab.research.google.com/github/kiyomi0917/DoSA-PJ/blob/main/KO_0623.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install Janome==0.3.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 12.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from janome.tokenizer import Tokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

# Load the Excel file
file_path = '/content/drive/MyDrive/dataset_0619.xlsx'
xl = pd.ExcelFile(file_path)

In [4]:
# Load the desired sheet into a DataFrame
df1 = xl.parse('0619データ')

#print(df1) # Print the DataFrame

In [5]:
# テキストを分割する関数_すべての単語を抽出する
t=Tokenizer()
def tokenize1(text):
    tokens = t.tokenize(text)
    noun = []
    for token in tokens:
        noun.append(token.surface)
    return noun

# 単語を品詞ごとに抽出
def tokenize2(text):
    tokens = t.tokenize(text)
    noun = []
    for token in tokens:
        # 「名詞」「動詞」「形容詞」「形容動詞」を取り出す
        if token.part_of_speech.split(',')[0] in ['名詞', '動詞', '形容詞', '形容動詞']:
            noun.append(token.surface)
    return noun


In [6]:
docs = df1["概要"]
labels = df1["ハラスメント判定"].astype(int)  # ラベルを整数に変換
print(labels)

0      0
1      0
2      0
3      0
4      0
      ..
108    1
109    1
110    1
111    1
112    1
Name: ハラスメント判定, Length: 113, dtype: int64


In [7]:
train_data = docs
train_labels =labels
train_labels = train_labels.astype(int)  # ラベルを整数に変換

In [8]:

#from sklearn.metrics import accuracy_score
file_path = '/content/drive/MyDrive/dataset_0619.xlsx'
xl = pd.ExcelFile(file_path)

#Load a sheet into a DataFrame by name
df2 = xl.parse('0620データ')

# テストデータの概要とラベルを取得
test_data = df2["概要"]
test_labels = df2["ハラスメント判定"]

# テキストデータをベクトル化するためのVectorizerを定義
vectorizer = TfidfVectorizer(tokenizer=tokenize2)

# ベクトル化に使用するトークン化関数（tokenize1）を指定してVectorizerを初期化
vectorizer = TfidfVectorizer(tokenizer=tokenize2)

# 学習データをベクトル化
train_matrix = vectorizer.fit_transform(train_data)

# モデルを設定
model = RandomForestClassifier()

# パラメータでモデルを再学習
model.fit(train_matrix, train_labels)

# 訓練データの予測
train_predictions = model.predict(train_matrix)

# 訓練データの正解率を計算して表示
train_accuracy = accuracy_score(train_labels, train_predictions)
print("訓練データの正解率:", train_accuracy)

# テストデータを変換
test_matrix = vectorizer.transform(test_data)

# テストデータの予測
test_predictions = model.predict(test_matrix)

# テストデータの正解率を計算して表示
test_accuracy = accuracy_score(test_labels, test_predictions)
print("テストデータの正解率:", test_accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


訓練データの正解率: 1.0
テストデータの正解率: 0.7727272727272727


In [9]:
# ハイパーパラメーターの値の候補を設定
model_param_set_grid = {"n_estimators": [50, 100, 200],
                        "max_depth": [None, 10, 20, 30],
                        "random_state": [42]}

# グリッドサーチでハイパーパラメーターを探索
clf = GridSearchCV(model, model_param_set_grid)
clf.fit(train_matrix, train_labels)

# 最適なパラメータとそのときのスコアを表示
print("ハイパーパラメーター:{}".format(clf.best_params_))
# print("ベストスコア:", clf.best_score_)

# 最適なパラメータでモデルを構築
best_model = RandomForestClassifier(**clf.best_params_)

# 最適なパラメータでモデルを再学習
best_model.fit(train_matrix, train_labels)

# 訓練データの予測
train_predictions = best_model.predict(train_matrix)

# 訓練データの正解率を計算して表示
train_accuracy = accuracy_score(train_labels, train_predictions)
print("訓練データの正解率:", train_accuracy)

# テストデータを変換/予測
test_matrix = vectorizer.transform(test_data)
test_predictions = best_model.predict(test_matrix)

# テストデータの正解率を計算して表示
test_accuracy = accuracy_score(test_labels, test_predictions)
print("テストデータの正解率:", test_accuracy)

ハイパーパラメーター:{'max_depth': None, 'n_estimators': 50, 'random_state': 42}
訓練データの正解率: 1.0
テストデータの正解率: 0.7727272727272727


In [10]:
#from sklearn import svm
import pickle

#学習モデルの保存
with open('model.pickle', mode='wb') as f:
    pickle.dump(best_model,f,protocol=2)
